This codes takes CIM Program data, CIM Assesssments Data, and Admissions data and returns effectiveness measure for TOC, COPD, and CHF Care Management Programs.

Instructions for how to update.
1. Run 3 SQL queries in zip folder - "progs", "assessments", and "admits", they are stored in github.com/tysonjens
2. Run queries and save locally to your computer. Delimit query results with "|".
3. Update "read_csv" commands (below) to corresopnd to your local paths
4. Run the script and note outputs for the several analyses


In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil.parser import parse
from pandas.tseries.offsets import Day
from sklearn import linear_model
from sklearn.preprocessing import Imputer
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pyodbc
# import statsmodels.formula.api as sm
# import warnings; warnings.simplefilter('ignore')
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 500)
%matplotlib inline

## Functions

Before importing and analyzing the data, we define several functions that help link and create features using the three data files.

In [ ]:
## function that finds the most recent discharge before a program begins
def find_index_discharge_date(programs, admissions):
    admissions = admissions.sort_values(by='discharge_date', ascending=False)
    index_dates = np.empty(programs.shape[0])
    index_dates[:] = np.nan
    index_dates = list(index_dates)
    for index, row in programs.iterrows():
        admit_pat = admissions[admissions['EMPI']==row['EMPI']]
        for index2, row2 in admit_pat.iterrows():
            if row2['discharge_date'] < row['prog_create_date']:
                index_dates[index] = row2['discharge_date']
                break
            else:
                continue
    return index_dates

In [ ]:
## function that finds the first assessment completed after discharge and records the date
def find_first_assess(programs, assessments):
    assessments = assessments.sort_values(by='ASES_DT', ascending=True)
    first_assess_dates = np.empty(programs.shape[0])
    first_assess_dates[:] = np.nan
    first_assess_dates = list(first_assess_dates)
    first_assess_name = np.empty(programs.shape[0])
    first_assess_name[:] = np.nan
    first_assess_name = list(first_assess_name)
    for index, row in programs.iterrows():
        assess_pat = assessments[assessments['EMPI']==row['EMPI']]
        for index2, row2 in assess_pat.iterrows():
            if row2['ASES_DT'] > row['index_date']:
                first_assess_dates[index] = row2['ASES_DT']
                first_assess_name[index] = row2['ASES_NM']
                break
            else:
                continue
    return first_assess_dates, first_assess_name

In [ ]:
## function that finds a LACE score that occurred within 40 days (prior) to program start date
def find_lace_prior_to_enroll(programs, lace, window_size=40):
    lace = lace.sort_values(by='ASES_DT', ascending=False)
    lace_scores = np.empty(programs.shape[0])
    lace_scores[:] = np.nan
    lace_scores = list(lace_scores)
    for index, row in programs.iterrows():
        lace_pat = lace[lace['EMPI']==row['EMPI']]
        for index2, row2 in lace_pat.iterrows():
            if (row2['ASES_DT'] < row['prog_create_date']) & (row2['ASES_DT'] > (row['prog_create_date']-timedelta(days=window_size))):
                lace_scores[index] = row2['ASES_SCOR']
                break
            else:
                continue
    return lace_scores

In [ ]:
## function that finds the LACE dates that occurred within 40 days (prior) to program start date
def find_lace_dt_prior_to_enroll(programs, lace, window_size=40):
    lace = lace.sort_values(by='ASES_DT', ascending=False)
    lace_dates = np.empty(programs.shape[0])
    lace_dates[:] = np.nan
    lace_dates = list(lace_dates)
    for index, row in programs.iterrows():
        lace_pat = lace[lace['EMPI']==row['EMPI']]
        for index2, row2 in lace_pat.iterrows():
            if (row2['ASES_DT'] < row['prog_create_date']) & (row2['ASES_DT'] > (row['prog_create_date']-timedelta(days=window_size))):
                lace_dates[index] = row2['ASES_DT']
                break
            else:
                continue
    return lace_dates

In [ ]:
## Function that counts assessments that occur during a program.
def find_assessments_during_program(programs, assess):
    assess_cnt = np.zeros(programs.shape[0])
    assess_cnt = list(assess_cnt)
    for index, row in programs.iterrows():
        assess_pat = assess[(assess['EMPI']==row['EMPI']) &
                             (assess['ASES_DT'] > (row['prog_create_date']-timedelta(days=1))) &
                             (assess['ASES_DT'] < (row['prog_end_date']+timedelta(days=1)))]
        assess_cnt[index] = assess_pat.shape[0]
    return assess_cnt

In [ ]:
## Function that counts assessments in the 30-day window following a discharge (starts at "index date")
def find_assessments_during_TOC(programs, assess):
    assess_cnt_toc = np.zeros(programs.shape[0])
    assess_cnt_toc = list(assess_cnt_toc)
    for index, row in programs.iterrows():
        assess_pat = assess[(assess['EMPI']==row['EMPI']) &
                             (assess['ASES_DT'] > (row['index_date'])) &
                             (assess['ASES_DT'] < (row['index_date']+timedelta(days=31)))]
        assess_cnt_toc[index] = assess_pat.shape[0]
    return assess_cnt_toc

In [ ]:
## Function that counts assessments during the first 90 days of a program. Used in the this code for DM programs.
def find_assessments_during_program_dm(programs, assess):
    assess_cnt = np.zeros(programs.shape[0])
    assess_cnt = list(assess_cnt)
    for index, row in programs.iterrows():
        assess_pat = assess[(assess['EMPI']==row['EMPI']) &
                             (assess['ASES_DT'] > (row['prog_create_date']-timedelta(days=1))) &
                             (assess['ASES_DT'] < (row['prog_create_date']+timedelta(days=91)))]
        assess_cnt[index] = assess_pat.shape[0]
    return assess_cnt

In [ ]:
## Convenience function that converts a list of counts into a binary "flag"
def list_of_counts_to_flag(ases):
    ases_yn = []
    for num in ases:
        if num>0:
            ases_yn.append(1)
        else:
            ases_yn.append(0)
    return ases_yn

In [ ]:
## Function that counts the number of admits and length of stay in a window AFTER a program begins. For disease management
def get_adm_after(programs, admissions, window_size=90):
    admits_in_window = list(np.zeros(programs.shape[0]))
    los_in_window = list(np.zeros(programs.shape[0]))
    for index, row in programs.iterrows():
        admit_pat = admissions[admissions['EMPI']==row['EMPI']]
        count = 0
        los = 0
        for index2, row2 in admit_pat.iterrows():
            if (row2['admit_date'] < (row['prog_create_date']+timedelta(days=window_size))) & (row2['admit_date'] > row['prog_create_date']):
                count+=1
                los += row2['length_of_stay']
        admits_in_window[index] = count
        los_in_window[index] = los
    return admits_in_window, los_in_window

In [ ]:
## Function that counts the number of admits and length of stay in a window BEFORE a program begins. For disease management.
def get_adm_before(programs, admissions, window_size=90):
    admits_in_window = list(np.zeros(programs.shape[0]))
    los_in_window = list(np.zeros(programs.shape[0]))
    for index, row in programs.iterrows():
        admit_pat = admissions[admissions['EMPI']==row['EMPI']]
        count = 0
        los = 0
        for index2, row2 in admit_pat.iterrows():
            if (row2['admit_date'] > (row['prog_create_date']-timedelta(days=window_size))) & (row2['admit_date'] > (row['prog_create_date']-timedelta(days=10))):
                count+=1
                los += row2['length_of_stay']
        admits_in_window[index] = count
        los_in_window[index] = los
    return admits_in_window, los_in_window

In [ ]:
## Function that counts the number of admits in a window AFTER the index readmission. For TOC.
def get_adm_after_index_discharge(programs, admissions, window_size=30):
    admits_in_window = list(np.zeros(programs.shape[0]))
    for index, row in programs.iterrows():
        admit_pat = admissions[admissions['EMPI']==row['EMPI']]
        count = 0
        for index2, row2 in admit_pat.iterrows():
            if (row2['admit_date'] < (row['index_date']+timedelta(days=window_size))) & (row2['admit_date'] > row['index_date']):
                count+=1
        admits_in_window[index] = count
    return admits_in_window

In [ ]:
## Convenience function that converts a column of integer counts into a binary "flag"
def adms_to_one_zero(progs, field):
    adm_yn = []
    for index, row in progs.iterrows():
        if row[field]>0:
            adm_yn.append(1)
        else:
            adm_yn.append(0)
    return adm_yn

In [ ]:
## Plots "Receiver Operating Characteristic" Curve to evaluate Classification Model Performance.  
## Takes "TPR" or true positive rates, and "FPR" for false positive rates for various thresholds and returns plot of curve
def plotroc(TPR, FPR):
    roc_auc = auc(TPR, FPR)
    plt.figure()
    lw = 2
    plt.plot(TPR, FPR, color='darkorange',
             lw=lw, label="ROC curve area = {0:0.4f}".format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
## Function that uses "bootstraping" to find the distribution of likely values for model coefficients.
def bootstrap_ci_coefficients(X_train, y_train, num_bootstraps, class_weight={0: 1, 1: 1}):
    bootstrap_estimates = []
    for i in np.arange(num_bootstraps):
        sample_index = np.random.choice(range(0, len(y_train)), len(y_train))
        X_samples = X_train[sample_index]
        y_samples = y_train[sample_index]
        lm = linear_model.LogisticRegression(class_weight=class_weight, penalty='l1', C=100000000, solver='liblinear')
        lm.fit(X_samples, y_samples)
        bootstrap_estimates.append(lm.coef_[0])
    bootstrap_estimates = np.asarray(bootstrap_estimates)
    return bootstrap_estimates

In [ ]:
## For COPD, find wehther or not there was a 90 day admission
def tag_copd_admissions(dm_progs, rel_admits):
    dm_admit_yn = list(np.zeros(rel_admits.shape[0]))
    dm_optin_yn = np.empty(rel_admits.shape[0])
    dm_optin_yn[:] = np.nan
    dm_optin_yn = list(dm_optin_yn)
    dm_week = np.empty(rel_admits.shape[0])
    dm_week[:] = np.nan
    dm_week = list(dm_optin_yn)
    for index, row in dm_progs.iterrows():
        for index2, row2 in rel_admits.iterrows():
            if (row2['EMPI']==row['EMPI']) & (row2['admit_date'] > (row['prog_create_date']-timedelta(days=90))) & (row2['admit_date'] < (row['prog_create_date']+timedelta(days=90))):
                dm_admit_yn[index2] = 1
                dm_optin_yn[index2] = (row['copd_ases']>0)
                dm_week[index2] = int(((row2['admit_date']-row['prog_create_date'])/timedelta(days=1))/7)
    return dm_admit_yn, dm_optin_yn, dm_week

In [ ]:
## For CHF, find wehther or not there was a 90 day admission
def tag_chf_admissions(dm_progs, rel_admits):
    dm_admit_yn = list(np.zeros(rel_admits.shape[0]))
    dm_optin_yn = np.empty(rel_admits.shape[0])
    dm_optin_yn[:] = np.nan
    dm_optin_yn = list(dm_optin_yn)
    dm_week = np.empty(rel_admits.shape[0])
    dm_week[:] = np.nan
    dm_week = list(dm_optin_yn)
    for index, row in dm_progs.iterrows():
        for index2, row2 in rel_admits.iterrows():
            if (row2['EMPI']==row['EMPI']) & (row2['admit_date'] > (row['prog_create_date']-timedelta(days=90))) & (row2['admit_date'] < (row['prog_create_date']+timedelta(days=90))):
                dm_admit_yn[index2] = 1
                dm_optin_yn[index2] = (row['chf_ases']>0)
                dm_week[index2] = int(((row2['admit_date']-row['prog_create_date'])/timedelta(days=1))/7)
    return dm_admit_yn, dm_optin_yn, dm_week

In [ ]:
## Convinience function that converts integers less that 30 to "1", else "0" to identify patient who received med_rec
def medrec_yn(medrec):
    medrec_yn = []
    medrec = list(medrec)
    for i in medrec:
        if i < 30:
            medrec_yn.append(1)
        else:
            medrec_yn.append(0)
    return medrec_yn
    

## Read in Data

In [ ]:
# with open('progs.sql', 'r') as myfile:
#     progs_sql_str=myfile.read().replace('\n', ' ')

In [ ]:
# cnxn_progs = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=CIM_RPT')

# progs = pd.read_sql(progs_sql_str, cnxn_progs)

# cnxn_progs.close()

In [ ]:
## READ In Programs, Admissions, and Assessments data
progs = pd.read_csv('../data/progs20190624.csv', sep='|', low_memory=False)

In [ ]:
## Remember to strip out SQL notifications from SQL results
admits = pd.read_csv('../data/admissions20190624.csv', sep='|', low_memory=False)

In [ ]:
## Note that "error_bad_lines=False" instructs this code to ignore observations that don't have the expected number of columns
assess = pd.read_csv('../data/assessments20190624.csv', sep='|', low_memory=False, error_bad_lines=False)

## Clean Data

#### Clean progs

In [ ]:
progs.shape[0]

In [ ]:
progs = progs.drop_duplicates()

In [ ]:
## python doesn't recognize dates as far as 2999, so replacing them with 2100
mem_end_date = {'2999-12-31 00:00:00.000':'2100-12-31 00:00:00.000',
               '2999-12-31 00:00:00':'2100-12-31 00:00:00.000'}
progs['End_Date'] = progs['End_Date'].replace(mem_end_date)

In [ ]:
#progs['End_Date'] = np.where(progs['End_Date'].year == 2999, progs['End_Date'].replace(year=2100), progs['End_Date'])

In [ ]:
## convert dates to date time
progs['prog_create_date'] = pd.to_datetime(progs['ASGN_TMS'])
progs['date_of_birth'] = pd.to_datetime(progs['DOB'])
progs['prog_end_date'] = pd.to_datetime(progs['END_TMS'])
progs['Eff_Date'] = pd.to_datetime(progs['Eff_Date'])
progs['End_Date'] = pd.to_datetime(progs['End_Date'])

In [ ]:
# ## create opt-in and opt-out with stop reason
# opt_in = {'Admit to Facility':1, 'Discharge to Custodial':1, 'Discharged to Hospice':1, 
#           'Discharged to PCP':1, 'Goals Met':1, 'Referred to other program':1, 
#           'Refused':0, 'Unable to Reach':0, 'Barriers to participation':0, 'Expired':np.nan, 
#          'Criteria not met':np.nan, 'Disenrolled from HP/Medical Group':np.nan, '6 months/1 year post transplant':np.nan,
#        'Pharmacy - physician recommended':np.nan}
# progs['is_optin'] = progs['PRGM_STOP_RSN'].replace(opt_in)

In [ ]:
## replace date_of_birth with age
progs['age'] = datetime.now() - progs['date_of_birth']
progs['age'] = progs['age'] / timedelta(days=1) / 365
progs['age'].fillna(float(progs['age'].mean()), inplace=True)

In [ ]:
## Replate "M"/"F" with 1/0
new_sex = {'F':0, 'M':1, 'U':1, 'I':0}
progs['is_male'] = progs['Sex'].replace(new_sex)
progs['is_male'].fillna(0, inplace=True)

In [ ]:
## drop unneeded columns
prog_cols_drop = ['PTNT_DK', 'DOB', 'Sex', 'ASGN_TMS', 'END_TMS', 'TNT_MKT_BK', 'date_of_birth', 'PRGM_STS', 'Patient_SK', 'CLNC_NM'
                 ,'Race','ZipCode', 'Current_Member', 'Deceased_Flag', 'HP_NM', 'PRIM_PRGM_FLAG']
progs = progs.drop(prog_cols_drop, axis=1)

In [ ]:
progs.shape[0]

In [ ]:
progs = progs.drop_duplicates()

In [ ]:
## Changes to program and assessment policies continued through July 1, 2018. 
## Subset programs after that date to keep things clean
progs = progs[(progs['prog_create_date']>'2018-07-01') & 
              (progs['prog_create_date']<(datetime.today()-timedelta(days=30)))]

In [ ]:
## Having subset dataframe, reset the index. This is so functions that use index work properly
progs = progs.reset_index()

In [ ]:
## Ensure programs and assessment EMPI are the same data type so they can be compared
progs['EMPI'] = progs['EMPI'].astype(float)

In [ ]:
## convert object columns to categoricals so they can be converted to dummy variables later
for col in ['RGON_NM', 'LOB_SUB_CGY', 'PRGM_NM', 'PRGM_STOP_RSN']:
    progs[col] = progs[col].astype('category')

#### Clean Admits

In [ ]:
## A few observations have "Nan" EMPIs. Convert to "999999999" to ensure functions work and they are ignored
admits['EMPI'].fillna(999999999, inplace=True)

admits['EMPI'] = admits['EMPI'].astype(int)

In [ ]:
## convert dates to date time
admits['admit_date'] = pd.to_datetime(admits['ACT_ADM_DT'])
admits['discharge_date'] = pd.to_datetime(admits['ACT_DISCH_DT'])

In [ ]:
## calculate length of stay using admit date and discharge date
admits['length_of_stay'] = ((admits['discharge_date']-admits['admit_date']) / np.timedelta64(1, 'D'))

In [ ]:
## drop unneeded columns
admits_cols_drop = ['REFERRAL_KEY', 'PATIENT', 'ACT_ADM_DT', 'ACT_DISCH_DT',
       'PriorAcuteDisch', 'PriorAcuteAdm', 'PriorSubDisch', 'PriorSubAdm',
       'DISCHARGE_DISPOSITION']
admits = admits.drop(admits_cols_drop, axis=1)

#### Clean Assessments

In [ ]:
## Remove assessments with weird filler date '2917-12-26 00:00:00.000'
assess = assess[assess['ASES_DT']!='2917-12-26 00:00:00.000']

In [ ]:
## Note January 30, 2019 - we began using "COMPLETION_DATE" instead of "ASES_DT", but in the query we have aliased 
## "COMPLETION DATE" as "ASES_DT" for code
assess['ASES_DT'] = pd.to_datetime(assess['ASES_DT'])

In [ ]:
assess['EMPI'] = assess['EMPI'].astype(float)

## Feature Engineering

In [ ]:
## for all programs, find the most recent discharge date (prior to program create date)
index_dates = find_index_discharge_date(progs, admits)
progs['index_date'] = index_dates
progs['index_date'] = pd.to_datetime(progs['index_date'])

In [ ]:
## subset all assessments to just LACE assessments - to be used below.
LACE = assess[assess['ASES_NM']=='LACE']

In [ ]:
## find lace scores that occurred recently (before program started) - 40 day window
lace_scores = find_lace_prior_to_enroll(progs, LACE)
progs['lace_score'] = lace_scores

In [ ]:
## find dates of most recent LACE score
lace_dates = find_lace_dt_prior_to_enroll(progs, LACE, window_size=1000)
progs['lace_date_prior_enroll'] = lace_dates

In [ ]:
## Calculate the number of days from the lace assessment to TOC program enrollment
progs['lace_to_enroll_days'] = (progs['prog_create_date']-progs['lace_date_prior_enroll'])/ timedelta(days=1)

In [ ]:
## find TOC assessments that matter for TOC
## 'MCG - Post-Hospitalization Follow-Up' was discontinued, but left in here to properly identify opt-in from historical data
assess_toc = assess[(assess['ASES_NM']=='MCG - Post-Hospitalization Follow-Up') |
                    (assess['ASES_NM']=='Post-Hospitalization Follow-up')
                  |  (assess['ASES_NM']=='Pharm - Post-Hospitalization Follow-up')
                    |  (assess['ASES_NM']=='Pharm ? Post-Hospitalization Follow-up')
                   ]

In [ ]:
## Subset assessments to Med Rec assessments
assess_medrec = assess[(assess['ASES_NM']=='CM Medication Reconciliation') |
                       (assess['ASES_NM']=='Med Reconciliation')]

In [ ]:
## Calculate the number of admits that occur within a 30-day window after program begins
thirty_day_after_adm, thirty_day_after_bd  = get_adm_after(progs, admits, window_size=30)
progs['adm_30_after'] = thirty_day_after_adm
progs['los_30_after'] = thirty_day_after_bd

In [ ]:
## Calculate the number of admits and length of stay that occur within 90 day window after program begins
ninety_day_after_adm, ninety_day_after_bd = get_adm_after(progs, admits, window_size=90)
progs['adm_90_after'] = ninety_day_after_adm
progs['los_90_after'] = ninety_day_after_bd

In [ ]:
## Calculate the number of admits and length of Stay that occur within 90 day window before program begins
ninety_day_before_adm , ninety_day_before_bd = get_adm_before(progs, admits)
progs['adm_90_before'] = ninety_day_before_adm
progs['los_90_before'] = ninety_day_before_bd

In [ ]:
## Calculate days from index discharge date to program create date, second line converts "datetime" format to integer
progs['time_to_enroll'] = progs['prog_create_date']-progs['index_date']
progs['time_to_enroll'] = progs['time_to_enroll']/ timedelta(days=1)

In [ ]:
## Calculate program duration
progs['prog_duration'] = (progs['prog_end_date'].fillna(datetime.today()))-progs['prog_create_date']
progs['prog_duration'] = progs['prog_duration']/ timedelta(days=1)

In [ ]:
## Find dates and names of first TOC assessment during program
toc_first_ases_dt, toc_first_ases_nm = find_first_assess(progs, assess_toc)

progs['toc_first_ases_dt'] = toc_first_ases_dt

In [ ]:
## Find dates and names of first med rec assessment during program
medrec_first_ases_dt, medrec_first_ases_nm = find_first_assess(progs, assess_medrec)

progs['medrec_first_ases_dt'] = medrec_first_ases_dt

In [ ]:
## Converts column to "datetime"
progs['toc_first_ases_dt'] = pd.to_datetime(progs['toc_first_ases_dt'])
progs['medrec_first_ases_dt'] = pd.to_datetime(progs['medrec_first_ases_dt'])

In [ ]:
## calculate time between index discharge and first relevant assessment or med rec
progs['index_to_first_ases'] = (progs['toc_first_ases_dt']-progs['index_date'])/timedelta(days=1)
progs['index_to_medrec'] = (progs['medrec_first_ases_dt']-progs['index_date'])/timedelta(days=1)

In [ ]:
## Create flag for whether med rec took place within 30 days after discharge
medrec_yn = medrec_yn(progs['index_to_medrec'])
progs['medrec_yn'] = medrec_yn

## Exploratory Data Analysis

In [ ]:
## Gut check - distribution of time to enroll should follow a skinny exponential distribution
## Most enrolls should happen in the first few days, then quickly tapers to long tail

fighist = plt.figure(figsize=(12,6))
ax1 = fighist.add_subplot(111)
ax1.set_title('Histogram, time to enroll post discharge')
ax1.hist(np.array(progs[progs['PRGM_NM']=='Transitions of Care - Post Discharge']['time_to_enroll'].dropna()), bins = 100, alpha = 0.4, density=1, label='TOC')
#ax1.hist(np.array(progs[progs['PRGM_NM']=='DM - CLD']['time_to_enroll'].dropna()), bins = 40, alpha = 0.6, density=1, label='CLD')
#ax1.hist(np.array(progs[progs['PRGM_NM']=='DM - HF']['time_to_enroll'].dropna()), bins = 40, alpha = 0.6, density=1, label='HF')
ax1.set_ylabel('Percent of Patients')
ax1.set_xlabel('days to enrollment')
ax1.legend();

## TOC - Feature Engineering

In [ ]:
progs[(progs['PRGM_NM']=='Transitions of Care - Post Discharge')].drop_duplicates().shape[0]

In [ ]:
## Subset programs list to only include active members and enrollments associated with a discharge.
## Also removes patients without a lace score OR whose program was "auto-closed" as a data clean up measure.
progs_toc_w_lace = progs[(progs['PRGM_NM']=='Transitions of Care - Post Discharge') &
                  ((progs['prog_create_date']-timedelta(days=30))>progs['Eff_Date']) &
                  ((progs['prog_create_date']-timedelta(days=30))<progs['End_Date']) &
                  ((progs['time_to_enroll']<30)) &
                  (progs['PRGM_STOP_RSN']!='Auto-Closed')  &
                  (progs['lace_score'].isna()==False) &  
                  (progs['prog_create_date']<(datetime.today()-timedelta(days=35)))
                 ].reset_index()

In [ ]:
progs_toc_w_lace

In [ ]:
## Identify assessments that occured during TOC window that interfere with the opt-out effect.
assess_that_muddy = assess[(assess['ASES_NM'] == 'Social Work Initial') |
      ##(assess['ASES_NM'] == 'MCG - DMG - Preventative Assessment') |
      ##(assess['ASES_NM'] == 'Social Work Outreach Encounter') |
      (assess['ASES_NM'] == 'MCG - DMG - High-Risk Pregnancy - Assessment') | 
      (assess['ASES_NM'] == 'Social Work Note') |
      (assess['ASES_NM'] == 'Transplant Notes') |
      (assess['ASES_NM'] == 'MCG - DMG - Diabetes, Adult - Knowledge of Condition and Treatment Plan') |
      (assess['ASES_NM'] == 'MCG - DMG - Complex Assessment') |
      ##(assess['ASES_NM'] == 'IVR Survey') |
      (assess['ASES_NM']=='MCG - Heart Failure - Knowledge of Condition and Treatment Plan') |
      (assess['ASES_NM']=='MCG - DMG - Heart Failure - Knowledge of Condition and Treatment Plan') |
      (assess['ASES_NM']=='MCG - Chronic Obstructive Pulmonary Disease - Knowledge of Condition and Treatment Plan') |
      (assess['ASES_NM']=='MCG - DMG - Chronic Lung Disease (CLD)') |
      (assess['ASES_NM'] == 'MCG - DMG - Heart Failure') |
      (assess['ASES_NM'] == 'MCG - DMG - CLD')].drop_duplicates(subset='PTNT_ASES_DK')

In [ ]:
## Count the number of interfering assessments that occur during a TOC program
assess_toc_cnt = find_assessments_during_TOC(progs_toc_w_lace, assess_that_muddy)
progs_toc_w_lace['assess_dur_TOC_cnt'] = assess_toc_cnt

In [ ]:
## create count and y/n columns for assessment that interfere
cm_toc_ases = find_assessments_during_program(progs_toc_w_lace, assess_toc)
cm_toc_ases_flag = list_of_counts_to_flag(cm_toc_ases)
progs_toc_w_lace['cnt_toc_cm_touch'] = cm_toc_ases
progs_toc_w_lace['toc_cm_touch_yn'] = cm_toc_ases_flag

In [ ]:
## Subset to only include patients that had no interfering assessments during their 30-day TOC window
toc_interfere_ind = adms_to_one_zero(progs_toc_w_lace, 'assess_dur_TOC_cnt')
progs_toc_w_lace['toc_interfere_ind'] = toc_interfere_ind

In [ ]:
## Clean opt-ins and opt-outs using the 'toc_interfere_ind'

progs_toc_w_lace = progs_toc_w_lace[progs_toc_w_lace['toc_interfere_ind']==0].reset_index(drop=True)

In [ ]:
## Counts readmission in 30-day TOC window
thirty_day_after_TOC = get_adm_after_index_discharge(progs_toc_w_lace, admits)
progs_toc_w_lace['adm_30_after_TOC'] = thirty_day_after_TOC

In [ ]:
## Converts counts of admission to binary flag
adm_yn = adms_to_one_zero(progs_toc_w_lace, 'adm_30_after_TOC')
progs_toc_w_lace['is_30_TOC_adm'] = adm_yn

In [ ]:
## Drop unneeded columns
admits_cols_drop = ['level_0', 'index', 'Eff_Date', 'End_Date', 'ASGN_USR', 'lace_date_prior_enroll',
       'adm_30_after', 'los_90_after', 'los_90_before', 'toc_first_ases_dt'
                   ,'medrec_first_ases_dt']
progs_toc_w_lace = progs_toc_w_lace.drop(admits_cols_drop, axis=1)

In [ ]:
# Give the filename you wish to save the file to
toc_regression_filename = '../data/progs_toc_20190517.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(toc_regression_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    progs_toc_w_lace.to_csv(toc_regression_filename)
else:
    print('WARNING: This file already exists!')

In [ ]:
## Create binary dummy columns for each value in categorical variables
progs_toc_ent = pd.get_dummies(pd.Series(list(progs_toc_w_lace['ENT_TYPE'])))
progs_toc_lob = pd.get_dummies(pd.Series(list(progs_toc_w_lace['LOB_SUB_CGY'])))
progs_toc_rgn = pd.get_dummies(pd.Series(list(progs_toc_w_lace['RGON_NM'])))

In [ ]:
## Merges dummy columns into dataframe
progs_toc_w_lace = pd.concat([progs_toc_w_lace, progs_toc_lob, progs_toc_ent, progs_toc_rgn], axis=1)

## TOC - Data Exploration

In [ ]:
progs_toc_w_lace[(progs_toc_w_lace['toc_cm_touch_yn']==0)]['los_30_after'].mean()

In [ ]:
## Find the distribution of length of stay for TOC patients

fighist = plt.figure(figsize=(12,6))
ax1 = fighist.add_subplot(111)
ax1.set_title('Histogram, length of stay')
ax1.hist(np.array(progs_toc_w_lace[(progs_toc_w_lace['toc_cm_touch_yn']==0)]['los_30_after'].dropna()), bins = 20, alpha = 0.4, density=1, label='TOC')
#ax1.hist(np.array(progs[progs['PRGM_NM']=='DM - CLD']['time_to_enroll'].dropna()), bins = 40, alpha = 0.6, density=1, label='CLD')
#ax1.hist(np.array(progs[progs['PRGM_NM']=='DM - HF']['time_to_enroll'].dropna()), bins = 40, alpha = 0.6, density=1, label='HF')
ax1.set_ylabel('Percent of Patients')
ax1.set_xlabel('length of stay, days')
ax1.legend();

In [ ]:
## Convenience code to explore correlation between one variable and all other variables
progs_toc_w_lace.corr()['prog_duration']

In [ ]:
## Convenience code to pivot readmission flag and opt-in / opt-out flag
progs_toc_w_lace.pivot_table(values='is_30_TOC_adm', columns='toc_cm_touch_yn', aggfunc=['mean', 'count'])

## TOC - Regression

In [ ]:
## In earlier versions classes were more imbalanced. We accounted for this by assigning more importance
## to the minority class. In later version, we kept each class's importance = 1.
class_weight={0: 1, 1: 1}

In [ ]:
## Selects explanatory variables for regression
x_cols = ['toc_cm_touch_yn', 'time_to_enroll', 'lace_score', 'prog_duration', 'IPA', 'MEDI-CAL', 'COMMERCIAL', 'is_male']

In [ ]:
## Subsets columns to relevant columns 
X = np.array(progs_toc_w_lace[x_cols])

In [ ]:
# ## Convenience Code to impute values for nulls in each column. 
# imput = Imputer(strategy='most_frequent')
# X = imput.fit_transform(X)

In [ ]:
## Creates array for "Y" - our target variable.
y_ad_yn = np.array(progs_toc_w_lace['is_30_TOC_adm'])

In [ ]:
## This code can split the dataset into training and test sets.
## In practice, we assign the entire dataset to training because we haven't used the model to predict
## This increases our n and helps find a signal for program effectiveness
X_ad_train, X_ad_test, y_ad_yn_train, y_ad_yn_test = train_test_split(X, y_ad_yn, test_size=.1, random_state=65)

In [ ]:
## Creates a logistic regression instance. C=100000000 reduces regularization to 0.
model_ad = linear_model.LogisticRegression(C=100000000, class_weight=class_weight, solver='liblinear')

In [ ]:
'I' in X_ad_train[:,7]

In [ ]:
## Fits model instance using our X and Y matricies
model_ad.fit(X_ad_train, y_ad_yn_train)

## TOC - Explore Regression Results

In [ ]:
## Having fit the model, we can now extract coefficients for X variables.
model_ad.coef_

In [ ]:
## Having fit the model, we can now extract the y intercept
model_ad.intercept_

In [ ]:
## Bootstrap likely distributions of values that each coefficient takes
ad_yn_bootstraps = bootstrap_ci_coefficients(X, y_ad_yn, 2000)
ad_yn_bootstraps = pd.DataFrame(ad_yn_bootstraps, columns=x_cols)

In [ ]:
## Plots distributions of likely values for each X variable
fig, axes = plt.subplots(2,4, figsize=(16,8))
col_names = x_cols

for m, ax in zip(col_names, axes.flatten()):
    ax.hist(ad_yn_bootstraps[m], bins=50)
    ax.plot([0, 0], [0, 155], color='red', linestyle='-', linewidth=2)
    ax.set_xlim(right=.5, left=-.5)
    ax.set_title(m)

In [ ]:
## Calculate Model Accuracy
r2 = model_ad.score(X, y_ad_yn)
print(r2)

In [ ]:
## Generate prediction probabilities and evaluate 
## predictiveness of model with AUC.
y_ad_yn_preds = model_ad.predict_proba(X_ad_test)[:,1]

In [ ]:
ad_TPR, ad_FPR, ad_thresholds = roc_curve(y_ad_yn_test, y_ad_yn_preds, pos_label=None, sample_weight=None, drop_intermediate=True)

In [ ]:
## Plot AUC, which helps determine the predictive value of the model
plotroc(ad_TPR, ad_FPR)

In [ ]:
## Generate predictions and evaluate predictiveness of model with AUC.
y_ad_yn_preds = model_ad.predict_proba(X)[:,1]

In [ ]:
y_ad_yn_preds_act = model_ad.predict(X)

In [ ]:
act_readmission_rate_optins = progs_toc_w_lace[progs_toc_w_lace['toc_cm_touch_yn']==1]['is_30_TOC_adm'].mean()
print(act_readmission_rate_optins)

In [ ]:
## find the actual readmission rate
act_readmission_rate_pop = y_ad_yn.mean()
print(act_readmission_rate_pop)

In [ ]:
## Create a mirror image of our X matrix
x1 = X

In [ ]:
## Set all opt-ins to opt-outs
x1[:,0] = 0

In [ ]:
## find what the model predicts the readmission likelihoods would be for each patient
y_ad_yn_preds_nooptins = model_ad.predict_proba(x1)[:,1]

In [ ]:
## FIND the expected readmission rate of the population
exp_readmission_rate_pop = y_ad_yn_preds_nooptins.mean()
print(exp_readmission_rate_pop)

In [ ]:
progs_toc_w_lace['is_30_TOC_adm_pred'] = y_ad_yn_preds_act

In [ ]:
pop_effect = (exp_readmission_rate_pop - act_readmission_rate_pop)/exp_readmission_rate_pop

In [ ]:
optin_effect = (progs_toc_w_lace.shape[0])*pop_effect/(progs_toc_w_lace[(progs_toc_w_lace['toc_cm_touch_yn']==1)].shape[0])
print(optin_effect)

In [ ]:
## use optin effect and actual optin readmission rate to infer optin readmission rate if there had be no optins
exp_readmission_rate_optins = act_readmission_rate_optins/(1-optin_effect)
print(exp_readmission_rate_optins)

In [ ]:
#['toc_cm_touch_yn', 'time_to_enroll', 'lace_score', 'prog_duration', 'IPA', 'MEDI-CAL', 'COMMERCIAL', 'is_male']

In [ ]:
## Theoretical effect of opting in on odds
exp_effect_odds = 1-np.exp(model_ad.coef_[0][0])
print(exp_effect_odds)

In [ ]:
conf_int_logodds = np.percentile(np.array(ad_yn_bootstraps['toc_cm_touch_yn']), [2.5,50,97.5])

#### Expected Effect Size

In [ ]:
conf_int_odds = 1-np.exp(np.percentile(np.array(ad_yn_bootstraps['toc_cm_touch_yn']), [2.5,50,97.5]))

In [ ]:
model_ad.coef_[0]

In [ ]:
coefs_exp = model_ad.coef_[0]

In [ ]:
coefs_exp

In [ ]:
## Experimental effect of opting in on odds
# Average Patient: Time to enroll is 6, lace score 10, prog duration 17, Medicare, male
odds_0 = np.exp(np.sum(np.array([0, 6, 10, 17, 1, 0, 0, 1])* coefs_exp.reshape(1,-1))+model_ad.intercept_[0])
odds_exp_1 = np.exp(np.sum(np.array([1, 6, 10, 17, 1, 0, 0, 1])* coefs_exp.reshape(1,-1))+model_ad.intercept_[0])
print(odds_0)
print(odds_exp_1)

In [ ]:
## Convert odds to probability for the opt-out case.  This is used for expected, high, and low calculations
proba_0 = odds_0/(1+odds_0)
print(proba_0)

In [ ]:
## Convert odds to probability for the expected case.
exp_proba_1 = odds_exp_1/(1+odds_exp_1)
print(exp_proba_1)

In [ ]:
## Calculate the reduction in readmission probability for the expected case
exp_effect_proba = (proba_0 - exp_proba_1)/proba_0
print('The most likely effect is a {:.1%} reduction in readmission likelihood.'.format(exp_effect_proba))

In [ ]:
## Calculate the absolute risk reduections for the expected case
exp_ARR = exp_readmission_rate_optins*exp_effect_proba
print(exp_ARR)

In [ ]:
print('The program is estimated to reduce admissions by {:.1f} for every 1,000 "Opt-ins".'.format(exp_ARR*1000))

#### High Effect Size

In [ ]:
coefs_high = model_ad.coef_[0]

In [ ]:
coefs_high[0] = conf_int_logodds[0]

In [ ]:
coefs_high

In [ ]:
## Experimental effect of opting in on odds
# Average Patient: Time to enroll is 6, lace score 10, prog duration 17, Medicare, male
high_odds_1 = np.exp(np.sum(np.array([1, 6, 10, 17, 1, 0, 0, 1])* coefs_high.reshape(1,-1))+model_ad.intercept_[0])

In [ ]:
high_proba_1 = high_odds_1/(1+high_odds_1)
print(high_proba_1)

In [ ]:
high_effect_proba = (proba_0 - high_proba_1)/proba_0
print('The highest likely effect is a {:.1%} reduction in readmission likelihood.'.format(high_effect_proba))

In [ ]:
high_ARR = exp_readmission_rate_optins*high_effect_proba
print(high_ARR)

#### Low Effect Size

In [ ]:
coefs_low = model_ad.coef_[0]

In [ ]:
coefs_low[0] = conf_int_logodds[2]

In [ ]:
coefs_low

In [ ]:
## Experimental effect of opting in on odds
# Average Patient: Time to enroll is 6, lace score 10, prog duration 17, Medicare, male
low_odds_1 = np.exp(np.sum(np.array([1, 6, 10, 17, 1, 0, 0, 1])* coefs_low.reshape(1,-1))+model_ad.intercept_[0])

In [ ]:
low_proba_1 = low_odds_1/(1+low_odds_1)
print(low_proba_1)

In [ ]:
low_effect_proba = (proba_0 - low_proba_1)/proba_0
print('The lowest likely effect is a {:.1%} reduction in readmission likelihood.'.format(low_effect_proba))

In [ ]:
low_ARR = exp_readmission_rate_optins*low_effect_proba
print(low_ARR)

In [ ]:
print('The 95% confidence interval ranges between {:.1%} and {:.1%}.'.format(high_effect_proba, low_effect_proba))

In [ ]:
coefs_low = coefs_exp

In [ ]:
## Number needed to treat

In [ ]:
print('The estimated NNT is {:.0f}, but could be as high as {:.0f} or as low as {:.0f}.'.format(
    1/exp_ARR, 1/low_ARR, 1/high_ARR))

# CHF - Data Cleaning

In [ ]:
## Create an "image" of all programs data set, specifically for CHF.
progs_chf = progs

In [ ]:
progs_chf.shape

In [ ]:
## Subset all programs to relevant CHF programs.
progs_chf = progs_chf[(progs_chf['PRGM_NM']=='DM - HF') &
                    ((progs_chf['prog_create_date']-timedelta(days=90))>progs_chf['Eff_Date']) &
                    ((progs_chf['prog_create_date']-timedelta(days=90))<progs_chf['End_Date']) &
                      (progs_chf['PRGM_STOP_RSN']!='Auto-Closed')  &
                      (progs_chf['prog_create_date']<(datetime.today()-timedelta(days=90)))].reset_index(drop=True)

In [ ]:
progs_chf.shape

In [ ]:
## Identify Assessments that occured during CHF window that interfere with the opt-out population.
assess_that_muddy_chf = assess[(assess['ASES_NM'] == 'Social Work Initial') |
      (assess['ASES_NM'] == 'Social Work Note') |
      (assess['ASES_NM'] == 'Transplant Notes') |
      (assess['ASES_NM'] == 'MCG - DMG - Diabetes, Adult - Knowledge of Condition and Treatment Plan') |
      (assess['ASES_NM'] == 'MCG - DMG - Complex Assessment') |
      ##(assess['ASES_NM'] == 'MCG - DMG - Preventative Assessment') |
      ##(assess['ASES_NM'] == 'Social Work Outreach Encounter') |
      (assess['ASES_NM'] == 'MCG - DMG - High-Risk Pregnancy - Assessment') | 
      (assess['ASES_NM'] == 'MCG - Heart Failure - Knowledge of Condition and Treatment Plan') | 
      (assess['ASES_NM'] == 'MCG - DMG - Heart Failure - Knowledge of Condition and Treatment Plan') | 
      ##(assess['ASES_NM'] == 'IVR Survey') |
      (assess['ASES_NM'] == 'MCG - DMG - CLD') |
      (assess['ASES_NM']=='MCG - Post-Hospitalization Follow-Up') | 
      (assess['ASES_NM']=='Post-Hospitalization Follow-up') |
      ##(assess['ASES_NM']=='TOC Post Discharge Outreach') | removed on 4/2 per discussion with Sameer/Adam
      (assess['ASES_NM']=='CM Medication Reconciliation') |
      (assess['ASES_NM']=='Med Reconciliation')].drop_duplicates(subset='PTNT_ASES_DK')

In [ ]:
## create assessments table for CHF
assess_chf_mhfk = assess[assess['ASES_NM']=='MCG - Heart Failure - Knowledge of Condition and Treatment Plan']
assess_chf_mdhf = assess[assess['ASES_NM']=='MCG - DMG - Heart Failure']
assess_chf_mdhfk = assess[assess['ASES_NM']=='MCG - DMG - Heart Failure - Knowledge of Condition and Treatment Plan']
frames = [assess_chf_mhfk, assess_chf_mdhf, assess_chf_mdhfk]
assess_chf = pd.concat(frames)

In [ ]:
## Count interfering assessments during program
assess_chf_cnt = find_assessments_during_program_dm(progs_chf, assess_that_muddy_chf)
progs_chf['assess_dur_chf_cnt'] = assess_chf_cnt

In [ ]:
## cnt chf related assessments during programs
cnt_chf_90 = find_assessments_during_program_dm(progs_chf, assess_chf)

In [ ]:
## Converts counts to flag indicating whether a patient opted in or out of the CHF program
progs_chf['chf_ases'] = cnt_chf_90
cm_chf_ases_yn = list_of_counts_to_flag(cnt_chf_90)
progs_chf['chf_ases_yn'] = cm_chf_ases_yn

In [ ]:
## Creates flag if program has an interfering assessment, subsets programs to those where flag==0
chf_interfere_ind = adms_to_one_zero(progs_chf, 'assess_dur_chf_cnt')
progs_chf['chf_interfere_ind'] = chf_interfere_ind

progs_chf = progs_chf[((progs_chf['chf_interfere_ind']==0) & (progs_chf['chf_ases_yn']==0)) |
                     (progs_chf['chf_ases_yn']==1)]

In [ ]:
progs_chf.reset_index(drop=True, inplace=True)

In [ ]:
progs_chf.shape

In [ ]:
## Identify the date and name of first CHF assessment within a CHF program
chf_first_ases_dt, chf_first_ases_nm = find_first_assess(progs_chf, assess_chf)

progs_chf['chf_first_ases_dt'] = chf_first_ases_dt
progs_chf['chf_first_ases_nm'] = chf_first_ases_nm

In [ ]:
progs_chf.shape

In [ ]:
# Give the filename you wish to save the file to
progs_chf_filename = '../data/progs_chf_20190331.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(progs_chf_filename)

In [ ]:
# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    progs_chf.to_csv(progs_chf_filename)
else:
    print('WARNING: This file already exists!')

## CHF - Explore

In [ ]:
progs_chf.pivot_table(values=['adm_90_after', 'adm_90_before'], index='chf_ases_yn', aggfunc=['mean', 'count', 'sum'], margins=True)

## Develop Dataset for Regression

In [ ]:
progs_chf_reg = progs_chf.drop(columns = ['index', 'RGON_NM', 'LOB_SUB_CGY', 'ENT_TYPE', 'Eff_Date',
       'End_Date', 'PRGM_NM', 'PRGM_STOP_RSN', 'ASGN_USR', 'prog_create_date',
       'prog_end_date', 'age', 'is_male', 'index_date', 'lace_score',
       'lace_date_prior_enroll', 'lace_to_enroll_days', 'adm_30_after',
       'los_30_after', 'los_90_after',
       'los_90_before', 'time_to_enroll', 'prog_duration', 'toc_first_ases_dt',
       'medrec_first_ases_dt', 'index_to_first_ases', 'index_to_medrec',
       'medrec_yn', 'assess_dur_chf_cnt', 'chf_ases',
       'chf_interfere_ind', 'chf_first_ases_dt', 'chf_first_ases_nm'])

In [ ]:
progs_chf_reg_2 = progs_chf_reg

In [ ]:
progs_chf_reg_3 = pd.concat([progs_chf_reg, progs_chf_reg_2], axis=0).reset_index()

In [ ]:
n = progs_chf_reg.shape[0]

In [ ]:
progs_chf_reg_3['Admits (1 before 0 after)'] = np.concatenate([np.zeros(n),np.ones(n)],axis=0)

In [ ]:
progs_chf_reg_3.shape

In [ ]:
progs_chf_reg_3['Admits'] = np.where(progs_chf_reg_3['Admits (1 before 0 after)'] == 0, 
                                    progs_chf_reg_3['adm_90_after'], progs_chf_reg_3['adm_90_before'])

In [ ]:
progs_chf_reg_3['Interaction'] = progs_chf_reg_3['chf_ases_yn'] * progs_chf_reg_3['Admits (1 before 0 after)']

In [ ]:
# Give the filename you wish to save the file to
chf_regression_filename = '../data/chf_regression_20190331.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(chf_regression_filename)

In [ ]:
# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    progs_chf_reg_3.to_csv(chf_regression_filename)
else:
    print('WARNING: This file already exists!')

#### Parallel Assumption Testing

In [ ]:
rel_empi_chf = list(progs_chf['EMPI'].unique())
rel_admits_chf = admits[admits['EMPI'].isin(rel_empi_chf)].reset_index(drop=True)

In [ ]:
len(rel_empi_chf)

In [ ]:
rel_admits_chf.head()

In [ ]:
rel_admits_chf.iloc[740:780]

In [ ]:
progs_chf.shape

In [ ]:
chf_admit_yn, chf_optin_yn, chf_week = tag_chf_admissions(progs_chf, rel_admits_chf)

In [ ]:
rel_admits_chf['chf_admit_yn'] = chf_admit_yn
rel_admits_chf['chf_optin_yn'] = chf_optin_yn
rel_admits_chf['chf_optin_week'] = chf_week

In [ ]:
rel_admits_chf_pivot = rel_admits_chf.pivot_table(values='chf_admit_yn', index='chf_optin_week', columns='chf_optin_yn', aggfunc='sum')

In [ ]:
# Give the filename you wish to save the file to
chf_parallel_filename = '../data/chf_parallel_20190331.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(chf_parallel_filename)

In [ ]:
# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    rel_admits_chf.pivot_table('chf_admit_yn', index='chf_optin_week', columns='chf_optin_yn', aggfunc='sum').to_csv(chf_parallel_filename)
else:
    print('WARNING: This file already exists!')

In [ ]:
rel_admits_chf_pivot = pd.read_csv(chf_parallel_filename)

In [ ]:
figparachf = plt.figure(figsize=(12,10))
ax4 = figparachf.add_subplot(111)
ax4.plot(rel_admits_chf_pivot['chf_optin_week'], rel_admits_chf_pivot['True'], color='b', label='Opt-ins')
ax4.plot(rel_admits_chf_pivot['chf_optin_week'], rel_admits_chf_pivot['False'], color='r', label='Opt-out')
#ax1.scatter(X_horses[:,0], X_horses[:,1], color='r', label='horses')
ax4.legend(shadow=True, fontsize='x-large')
ax4.set_xticks(rel_admits_chf_pivot['chf_optin_week'], minor=True)
##ax4.set_xticklabels(rel_admits_chf_pivot['chf_optin_week'], minor=True)
##ax1.set_xlabel('Weight (lb)',fontsize=font_size)
##ax1.set_ylabel('Height (in)',fontsize=font_size)
ax4.set_title('CHF - Parallel Trend',fontsize='xx-large')

## COPD - Data Cleaning

In [ ]:
progs_copd = progs

In [ ]:
progs_copd.shape

In [ ]:
progs_copd = progs_copd[(progs_copd['PRGM_NM']=='DM - CLD') & 
                      ((progs_copd['prog_create_date']-timedelta(days=90))>progs_copd['Eff_Date']) &
                      ((progs_copd['prog_create_date']-timedelta(days=90))<progs_copd['End_Date']) &
                        (progs_copd['PRGM_STOP_RSN']!='Auto-Closed')  &
                        (progs_copd['prog_create_date']<(datetime.today()-timedelta(days=90)))].reset_index(drop=True)

In [ ]:
## Identify Assessments that occured during COPD window that interfere with the opt-out effect.
assess_that_muddy_copd = assess[(assess['ASES_NM'] == 'Social Work Initial') |
      ##(assess['ASES_NM'] == 'MCG - DMG - Preventative Assessment') |
      ##(assess['ASES_NM'] == 'Social Work Outreach Encounter') |
      (assess['ASES_NM'] == 'MCG - DMG - High-Risk Pregnancy - Assessment') | 
      ##(assess['ASES_NM'] == 'IVR Survey') |
      (assess['ASES_NM'] == 'MCG - DMG - Heart Failure') |                         
      (assess['ASES_NM']=='MCG - Post-Hospitalization Follow-Up') | 
      (assess['ASES_NM'] == 'Social Work Note') |
      (assess['ASES_NM'] == 'Transplant Notes') |
      (assess['ASES_NM'] == 'MCG - DMG - Diabetes, Adult - Knowledge of Condition and Treatment Plan') |
      (assess['ASES_NM'] == 'MCG - DMG - Complex Assessment') |
      (assess['ASES_NM']=='Post-Hospitalization Follow-up') |
      ##(assess['ASES_NM']=='TOC Post Discharge Outreach') | removed on 4/2 per discussion with Sameer and Adam
      (assess['ASES_NM']=='CM Medication Reconciliation') |
      (assess['ASES_NM']=='Med Reconciliation')].drop_duplicates(subset='PTNT_ASES_DK')

In [ ]:
## create assessments table for COPD

assess_copd_mcopdk = assess[assess['ASES_NM']=='MCG - Chronic Obstructive Pulmonary Disease - Knowledge of Condition and Treatment Plan'].drop_duplicates(subset='PTNT_ASES_DK')
assess_copd_mdchr = assess[assess['ASES_NM']=='MCG - DMG - Chronic Lung Disease (CLD)'].drop_duplicates(subset='PTNT_ASES_DK')
assess_copd_mdcld = assess[assess['ASES_NM']=='MCG - DMG - CLD'].drop_duplicates(subset='PTNT_ASES_DK')
frames = [assess_copd_mcopdk, assess_copd_mdchr, assess_copd_mdcld]
assess_copd = pd.concat(frames)

In [ ]:
assess_copd_cnt = find_assessments_during_program_dm(progs_copd, assess_that_muddy_copd)
progs_copd['assess_dur_copd_cnt'] = assess_copd_cnt

In [ ]:
progs_copd.shape

In [ ]:
## cnt copd related assessments during programs
cnt_copd_90 = find_assessments_during_program_dm(progs_copd, assess_copd)

In [ ]:
progs_copd['copd_ases'] = cnt_copd_90
cm_copd_ases_yn = list_of_counts_to_flag(cnt_copd_90)

In [ ]:
progs_copd['copd_ases_yn'] = cm_copd_ases_yn

In [ ]:
copd_first_ases_dt, copd_first_ases_nm = find_first_assess(progs_copd, assess_copd)

progs_copd['copd_first_ases_dt'] = copd_first_ases_dt
progs_copd['copd_first_ases_nm'] = copd_first_ases_nm

In [ ]:
copd_interfere_ind = adms_to_one_zero(progs_copd, 'assess_dur_copd_cnt')
progs_copd['copd_interfere_ind'] = copd_interfere_ind

progs_copd = progs_copd[((progs_copd['copd_interfere_ind']==0) & (progs_copd['copd_ases_yn']==0)) |
                     (progs_copd['copd_ases_yn']==1)]

In [ ]:
progs_copd.shape

In [ ]:
progs_copd.reset_index(drop=True, inplace=True)

In [ ]:
# Give the filename you wish to save the file to
progs_copd_filename = '../data/progs_copd_20190331.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(progs_copd_filename)

In [ ]:
# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    progs_copd.to_csv(progs_copd_filename)
else:
    print('WARNING: This file already exists!')

## COPD - Explore

In [ ]:
progs_copd.pivot_table(values=['adm_90_after', 'adm_90_before'], index='copd_ases_yn', aggfunc=['mean', 'count', 'sum'], margins=True)

## Develop Dataset for Regression

In [ ]:
progs_copd_reg = progs_copd.drop(columns = ['index', 'RGON_NM', 'LOB_SUB_CGY', 'ENT_TYPE', 'Eff_Date',
       'End_Date', 'PRGM_NM', 'PRGM_STOP_RSN', 'ASGN_USR', 'prog_create_date',
       'prog_end_date', 'age', 'is_male', 'index_date', 'lace_score',
       'lace_date_prior_enroll', 'lace_to_enroll_days', 'adm_30_after',
       'los_30_after', 'los_90_after',
       'los_90_before', 'time_to_enroll', 'prog_duration', 'toc_first_ases_dt',
       'medrec_first_ases_dt', 'index_to_first_ases', 'index_to_medrec',
       'medrec_yn', 'assess_dur_copd_cnt', 'copd_ases',
       'copd_interfere_ind', 'copd_first_ases_dt', 'copd_first_ases_nm'])

In [ ]:
progs_copd_reg_2 = progs_copd_reg

In [ ]:
progs_copd_reg_3 = pd.concat([progs_copd_reg, progs_copd_reg_2], axis=0).reset_index()

In [ ]:
n1 = progs_copd_reg.shape[0]

In [ ]:
progs_copd_reg_3['Admits (1 before 0 after)'] = np.concatenate([np.zeros(n1),np.ones(n1)],axis=0)

In [ ]:
progs_copd_reg_3['Admits'] = np.where(progs_copd_reg_3['Admits (1 before 0 after)'] == 0, 
                                    progs_copd_reg_3['adm_90_after'], progs_copd_reg_3['adm_90_before'])

In [ ]:
progs_copd_reg_3['Interaction'] = progs_copd_reg_3['copd_ases_yn'] * progs_copd_reg_3['Admits (1 before 0 after)']

In [ ]:
# Give the filename you wish to save the file to
copd_regression_filename = '../data/copd_regression_20190331.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(copd_regression_filename)

In [ ]:
# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    progs_copd_reg_3.to_csv(copd_regression_filename)
else:
    print('WARNING: This file already exists!')

#### Parallel Assumption Testing

In [ ]:
rel_empi = list(progs_copd['EMPI'].unique())
rel_admits = admits[admits['EMPI'].isin(rel_empi)].reset_index(drop=True)

In [ ]:
copd_admit_yn, copd_optin_yn, copd_week = tag_copd_admissions(progs_copd, rel_admits)

In [ ]:
rel_admits['copd_admit_yn'] = copd_admit_yn
rel_admits['copd_optin_yn'] = copd_optin_yn
rel_admits['copd_optin_week'] = copd_week

In [ ]:
rel_admits.pivot_table('copd_admit_yn', index='copd_optin_week', columns='copd_optin_yn', aggfunc='sum')

In [ ]:
# Give the filename you wish to save the file to
copd_parallel_filename = '../data/copd_parallel_20190331.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(copd_parallel_filename)

In [ ]:
# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    rel_admits.pivot_table('copd_admit_yn', index='copd_optin_week', columns='copd_optin_yn', aggfunc='sum').to_csv(copd_parallel_filename)
else:
    print('WARNING: This file already exists!')

In [ ]:
rel_admits_copd_pivot = pd.read_csv(copd_parallel_filename)

In [ ]:
rel_admits_copd_pivot

In [ ]:
figparacopd = plt.figure(figsize=(12,10))
ax5 = figparacopd.add_subplot(111)
ax5.plot(rel_admits_copd_pivot['copd_optin_week'], rel_admits_copd_pivot['True'], color='b', label='Opt-ins')
ax5.plot(rel_admits_copd_pivot['copd_optin_week'], rel_admits_copd_pivot['False'], color='r', label='Opt-out')
#ax1.scatter(X_horses[:,0], X_horses[:,1], color='r', label='horses')
ax5.legend(shadow=True, fontsize='x-large')
ax5.set_xticks(rel_admits_copd_pivot['copd_optin_week'], minor=True)
##ax4.set_xticklabels(rel_admits_chf_pivot['chf_optin_week'], minor=True)
##ax1.set_xlabel('Weight (lb)',fontsize=font_size)
##ax1.set_ylabel('Height (in)',fontsize=font_size)
ax5.set_title('COPD - Parallel Trend',fontsize='xx-large')
plt.show()